<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/013_Missing_Data_Imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧹 Obsługa Brakujących Danych (Missing Data)

W prawdziwym świecie dane nigdy nie są czyste. Zawsze czegoś brakuje:
*   Klient nie podał wieku.
*   Awaria czujnika temperatury.
*   Błąd zapisu w bazie.

W Pythonie braki oznaczamy jako **NaN** (Not a Number).
Większość modeli ML (Regresja, Sieci Neuronowe) **wywali się**, jeśli zobaczy NaN.

Mamy trzy główne strategie:
1.  **Usuwanie:** "Nie ma danych? Do śmieci." (Ryzykowne).
2.  **Imputacja Prosta:** "Wstaw średnią." (Szybkie, ale zniekształca).
3.  **Imputacja Inteligentna:** "Zgadnij wartość na podstawie grupy." (Najlepsze).

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# 1. TWORZYMY BRUDNE DANE (Symulacja HR)
# Mamy pracowników. Niektórzy nie podali WIEKU, inni PENSJI.
data = {
    'Stanowisko': ['Junior', 'Senior', 'Junior', 'Senior', 'Junior', 'Senior', 'Boss'],
    'Wiek':       [22,       45,       np.nan,   50,       23,       np.nan,   55],
    'Pensja':     [4000,     12000,    3500,     np.nan,   np.nan,   13000,    25000]
}

df = pd.DataFrame(data)

print("--- ORYGINAŁ (Zauważ NaN) ---")
display(df)

# Sprawdźmy, ile dokładnie brakuje
print("\nIle braków w każdej kolumnie?")
print(df.isnull().sum())

--- ORYGINAŁ (Zauważ NaN) ---


,Stanowisko,Wiek,Pensja
0,Junior,22.0,4000.0
1,Senior,45.0,12000.0
2,Junior,NaN,3500.0
3,Senior,50.0,NaN
4,Junior,23.0,NaN
5,Senior,NaN,13000.0
6,Boss,55.0,25000.0



Ile braków w każdej kolumnie?
Stanowisko    0
Wiek          2
Pensja        2
dtype: int64


## Metoda 1: Usuwanie (The Nuclear Option)

Najprostsze podejście: `dropna()`.
Usuwamy każdy wiersz, który ma chociaż jedną dziurę.

In [2]:
df_dropped = df.dropna()

print(f"Oryginalny rozmiar: {df.shape}")
print(f"Po usunięciu braków: {df_dropped.shape}")
print("-" * 30)

print("--- WYNIK ---")
display(df_dropped)

print("\nWNIOSEK: Straciliśmy prawie połowę danych! (3 z 7 wierszy).")
print("Jeśli masz mało danych, to samobójstwo.")

Oryginalny rozmiar: (7, 3)
Po usunięciu braków: (3, 3)
------------------------------
--- WYNIK ---


,Stanowisko,Wiek,Pensja
0,Junior,22.0,4000.0
1,Senior,45.0,12000.0
6,Boss,55.0,25000.0



WNIOSEK: Straciliśmy prawie połowę danych! (3 z 7 wierszy).
Jeśli masz mało danych, to samobójstwo.


## Metoda 2: Prosta Imputacja (Średnia)

Zamiast usuwać, wstawmy średnią z całej kolumny.
Użyjemy do tego `SimpleImputer` z biblioteki Scikit-Learn.

**Problem:**
Średnia pensja w firmie to np. 10 000.
Jeśli wstawimy tę liczbę Juniorowi (który zarabia 4000), to sztucznie go "wzbogacimy". Model zgłupieje.

In [3]:
# Tworzymy Imputer (Strategia: Średnia)
imputer = SimpleImputer(strategy='mean')

# Kopiujemy dane, żeby nie zepsuć oryginału
df_mean = df.copy()

# Wybieramy tylko kolumny liczbowe (Stanowiska nie da się uśrednić matematycznie)
cols_num = ['Wiek', 'Pensja']
df_mean[cols_num] = imputer.fit_transform(df[cols_num])

print("--- IMPUTACJA ŚREDNIĄ ---")
display(df_mean)
print("Spójrz na wiersz 4 (Junior). Dostał pensję ok. 11 500.")
print("To nie ma sensu! Juniorzy tyle nie zarabiają.")

--- IMPUTACJA ŚREDNIĄ ---


,Stanowisko,Wiek,Pensja
0,Junior,22.0,4000.0
1,Senior,45.0,12000.0
2,Junior,39.0,3500.0
3,Senior,50.0,11500.0
4,Junior,23.0,11500.0
5,Senior,39.0,13000.0
6,Boss,55.0,25000.0


Spójrz na wiersz 4 (Junior). Dostał pensję ok. 11 500.
To nie ma sensu! Juniorzy tyle nie zarabiają.


## Metoda 3: Imputacja Kontekstowa (Grupowanie)

To jest **sprytne podejście**.
Zamiast brać średnią wszystkich ludzi, weźmy:
*   Dla brakującego wieku Juniora -> Średnią wieku **innych Juniorów**.
*   Dla brakującej pensji Seniora -> Średnią pensję **innych Seniorów**.

Użyjemy do tego magii Pandasa: `groupby() + transform()`.

In [4]:
df_smart = df.copy()

# 1. Wypełniamy PENSJĘ średnią w ramach STANOWISKA
df_smart['Pensja'] = df_smart['Pensja'].fillna(
    df_smart.groupby('Stanowisko')['Pensja'].transform('mean')
)

# 2. Wypełniamy WIEK średnią w ramach STANOWISKA
df_smart['Wiek'] = df_smart['Wiek'].fillna(
    df_smart.groupby('Stanowisko')['Wiek'].transform('mean')
)

print("--- IMPUTACJA INTELIGENTNA (Grupowa) ---")
display(df_smart)

print("ANALIZA:")
print("- Junior (wiersz 4) dostał pensję 3750 (średnia Juniorów). Bardzo realne!")
print("- Senior (wiersz 3) dostał pensję 12500 (średnia Seniorów). Też realne.")

--- IMPUTACJA INTELIGENTNA (Grupowa) ---


,Stanowisko,Wiek,Pensja
0,Junior,22.0,4000.0
1,Senior,45.0,12000.0
2,Junior,22.5,3500.0
3,Senior,50.0,12500.0
4,Junior,23.0,3750.0
5,Senior,47.5,13000.0
6,Boss,55.0,25000.0


ANALIZA:
- Junior (wiersz 4) dostał pensję 3750 (średnia Juniorów). Bardzo realne!
- Senior (wiersz 3) dostał pensję 12500 (średnia Seniorów). Też realne.


## 🧠 Podsumowanie: Nie kłam danymi

Dlaczego sposób łatania dziur ma znaczenie?

**Tu jest haczyk.**
Każda imputacja to trochę **kłamstwo**. Wymyślamy dane, których nie było.
*   Jeśli usuniesz dane -> Wprowadzasz **Bias** (np. usuwasz tylko biednych ludzi, bo oni rzadziej podają zarobki, więc Twój model myśli, że świat jest bogaty).
*   Jeśli wstawisz średnią globalną -> Zabijasz różnorodność (**Variance**). Wszyscy stają się "przeciętni".
*   Jeśli użyjesz kontekstu (Grupy) -> Twoje "kłamstwo" jest najbardziej prawdopodobne.

**Zasada AI Engineera:**
Zanim wpiszesz `fillna(0)` albo `dropna()`, zastanów się przez minutę: *"Dlaczego tej wartości brakuje i co najbardziej przypomina prawdę?"*.